# 1. EXTRACCION DE NOTICIAS NYT

In [ ]:
# -*- coding: utf-8 -*-
"""
NYT – Amazon COMPANY (2020-07..2025-07)
- Incluye si 'amazon'/'aws'/'amzn' aparece en titular/resumen
- Excluye secciones/desks de cultura/estilo y tipos no-noticia
- Filtra 'selva Amazónica' salvo marcadores corporativos
- Blue Origin solo si hay Kuiper / “Amazon satellite”
- Muestra conteo mensual y guarda clean + audit
"""

import os
import re
import time
import requests
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

# ===== Parámetros =====
# Clave eliminada por seguridad para publicación en repositorio
NYT_API_KEY = "TU_API_KEY_AQUI"  

START_DATE = date(2020, 7, 1)   # 01-07-2020
END_DATE   = date(2025, 7, 31)  # 31-07-2025

REQUEST_DELAY_S = 8
MAX_REINTENTOS = 5
GUARDAR_POR_MES = False

# Palabras clave en titular/resumen
TITLE_ABSTRACT_REGEX = re.compile(r"\b(amazon|aws|amzn)\b", re.IGNORECASE)

# Exclusiones por sección/desk
HARD_EXCLUDE_SECTIONS = {
    "Arts", "Arts&Leisure", "Books", "Movies", "Style", "Theater", "Podcasts"
}
HARD_EXCLUDE_DESKS = {
    "Arts", "Arts&Leisure", "Books", "Culture", "Podcasts", "Styles", "Theater", "Weekend"
}

# Tipos a excluir
HARD_EXCLUDE_TYPES = {
    "briefing", "interactive feature", "obituary", "obit", "quote", "review", "video"
}

# Pistas de selva Amazónica
AMAZON_RAINFOREST_HINTS = {
    "rainforest", "rain forest", "river", "río", "indigenous", "tribe", "tribes", "indígena", "indígenas",
    "deforestation", "reforestation", "wildfire", "wildfires", "megafire", "megafires",
    "drought", "droughts", "ecuador", "peru", "brazil", "amazonia", "amazonas", "colombia",
    "fires", "heat", "water level", "low water", "dried up"
}

# Marcadores corporativos
CORPORATE_MARKERS = {
    "aws", "prime", "warehouse", "warehouses", "fulfillment", "delivery", "logistics", "retail",
    "nlrb", "osha", "teamsters", "union", "labor", "strike", "workers", "workforce", "employment",
    "marketplace", "merchant", "third-party sellers", "advertising", "cloud", "anthropic", "kuiper",
    "satellite", "data center", "datacenter", "ai", "earnings", "revenue", "acquisition", "roomba",
    "ring", "whole foods", "twitch", "zoox"
}

# Blue Origin: permitido solo con Kuiper/satélite Amazon
BLUE_ORIGIN_ALLOW_IF = {"kuiper", "amazon satellite", "amazon’s satellite", "project kuiper"}

# ===== Helpers =====
def _norm(s): return (s or "").strip()
def _lc(s):   return (s or "").lower()

def _type_of_material(doc):
    t = _lc(doc.get("type_of_material"))
    if not t:
        t = _lc(doc.get("document_type"))
    return t

def _news_desk(doc):    return _norm(doc.get("news_desk"))
def _section_name(doc): return _norm(doc.get("section_name"))
def _headline(doc):     return _norm(doc.get("headline", {}).get("main"))
def _abstract(doc):     return _norm(doc.get("abstract"))
def _lead_paragraph(doc): return _norm(doc.get("lead_paragraph"))

def _byline(doc):
    by = doc.get("byline") or {}
    if isinstance(by, dict) and by.get("original"):
        return _norm(by.get("original"))
    return ""

def matches_title_or_abstract(doc):
    text = f"{_headline(doc)} || {_abstract(doc)}"
    return bool(TITLE_ABSTRACT_REGEX.search(text))

def hard_excluded_section_or_desk(doc):
    sec, desk = _section_name(doc), _news_desk(doc)
    if sec in HARD_EXCLUDE_SECTIONS:
        return f"hard_excluded_section_or_desk({sec})"
    if desk in HARD_EXCLUDE_DESKS:
        return f"hard_excluded_section_or_desk({desk})"
    return ""

def hard_excluded_type(doc):
    t = _type_of_material(doc)
    if t in HARD_EXCLUDE_TYPES:
        return f"hard_excluded_type({t.title()})"
    return ""

def is_blue_origin_allowed(text_lc: str) -> bool:
    if "blue origin" in text_lc:
        return any(k in text_lc for k in BLUE_ORIGIN_ALLOW_IF)
    return True

def rainforest_context_without_corp(text_lc: str) -> bool:
    has_rainforest = any(h in text_lc for h in AMAZON_RAINFOREST_HINTS)
    if not has_rainforest:
        return False
    has_corp = any(k in text_lc for k in CORPORATE_MARKERS)
    return not has_corp

def should_include(doc):
    if not matches_title_or_abstract(doc):
        return False, "no_amazon_in_title_or_abstract"
    w = hard_excluded_section_or_desk(doc)
    if w: return False, w
    w = hard_excluded_type(doc)
    if w: return False, w
    text_lc = _lc(" ".join([_headline(doc), _abstract(doc), _lead_paragraph(doc), _byline(doc)]))
    if rainforest_context_without_corp(text_lc):
        return False, "amazon_rainforest_context"
    if not is_blue_origin_allowed(text_lc):
        return False, "blue_origin_without_kuiper"
    return True, ""

def to_row(doc):
    return {
        "Fecha":   pd.to_datetime(doc.get("pub_date"), errors="coerce"),
        "Titular": _headline(doc),
        "Resumen": _abstract(doc),
        "URL":     _norm(doc.get("web_url")),
        "Seccion": _section_name(doc),
        "Desk":    _news_desk(doc),
        "Tipo":    (_type_of_material(doc) or "").title(),
        "Autor":   _byline(doc),
    }

# ===== Cliente NYT =====
def fetch_archive(year: int, month: int, api_key: str, base_delay: int = REQUEST_DELAY_S):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json"
    params = {"api-key": api_key}
    tries = 0
    while True:
        tries += 1
        try:
            time.sleep(base_delay)
            r = requests.get(url, params=params, timeout=60)
            if r.status_code == 429:
                wait = min(base_delay * (2 ** (tries - 1)), 120)
                print(f"  -> 429 Too Many Requests. Backoff {wait}s (intento {tries}/{MAX_REINTENTOS})")
                time.sleep(wait)
                if tries >= MAX_REINTENTOS:
                    r.raise_for_status()
                continue
            r.raise_for_status()
            return r.json()
        except requests.exceptions.RequestException as e:
            if tries >= MAX_REINTENTOS:
                raise
            wait = min(base_delay * (2 ** (tries - 1)), 120)
            print(f"  -> Error {year}-{month:02d}: {e}. Reintentando en {wait}s…")
            time.sleep(wait)

# ===== Iteración mensual =====
def iter_months(start_date: date, end_date: date):
    cur = date(start_date.year, start_date.month, 1)
    endm = date(end_date.year, end_date.month, 1)
    while cur <= endm:
        yield cur.year, cur.month
        cur = cur + relativedelta(months=1)

# ===== Pipeline =====
def main():
    assert NYT_API_KEY and NYT_API_KEY != "TU_API_KEY_AQUI", "Falta NYT_API_KEY"

    start_month_label = f"{START_DATE.year}-{START_DATE.month:02d}"
    end_month_label   = f"{END_DATE.year}-{END_DATE.month:02d}"

    print("="*60)
    print(f"NYT Amazon COMPANY – Rango: {start_month_label} .. {end_month_label}")
    print("="*60)

    incluidos, audit, monthly_counts = [], [], {}

    for (y, m) in iter_months(START_DATE, END_DATE):
        etiqueta = f"{y}-{m:02d}"
        print(f"Procesando {etiqueta} …")
        try:
            data = fetch_archive(y, m, NYT_API_KEY)
        except Exception as e:
            print(f"  -> ERROR FATAL en {etiqueta}: {e}")
            continue

        docs = data.get("response", {}).get("docs", [])
        mes_incluidos = 0

        for doc in docs:
            row = to_row(doc)
            inc, why = should_include(doc)
            if inc:
                incluidos.append(row)
                mes_incluidos += 1
            else:
                ar = dict(row)
                ar["why_excluded"] = why
                audit.append(ar)

        monthly_counts[(y, m)] = mes_incluidos
        print(f"  -> {etiqueta}: INCLUIDOS {mes_incluidos}")

        if GUARDAR_POR_MES:
            if mes_incluidos:
                dfm = pd.DataFrame([r for r in incluidos if r["Fecha"].year == y and r["Fecha"].month == m])
                if not dfm.empty:
                    outm = f"nyt_amazon_company_{y}_{m:02d}_clean.csv"
                    dfm.sort_values("Fecha", ascending=False).to_csv(outm, index=False, encoding="utf-8-sig")
            dfa = pd.DataFrame([r for r in audit if isinstance(r.get("Fecha"), pd.Timestamp)
                                and r["Fecha"].year == y and r["Fecha"].month == m])
            if not dfa.empty:
                outa = f"nyt_amazon_company_{y}_{m:02d}_audit.csv"
                dfa.sort_values("Fecha", ascending=False).to_csv(outa, index=False, encoding="utf-8-sig")

    # Guardado final
    df_clean = pd.DataFrame(incluidos)
    if not df_clean.empty:
        df_clean = df_clean.dropna(subset=["Fecha"]).drop_duplicates(subset=["URL"]).sort_values("Fecha", ascending=False)
    df_audit = pd.DataFrame(audit)
    if not df_audit.empty:
        df_audit = df_audit.dropna(subset=["Fecha"]).sort_values("Fecha", ascending=False)

    out_clean = "FINAL_NYT_NEWS_5_ANIOS.CSV"  # Resultado
    out_audit = f"nyt_amazon_company_{start_month_label}_to_{end_month_label}_audit.csv"   # Bruto
    df_clean.to_csv(out_clean, index=False, encoding="utf-8-sig")
    df_audit.to_csv(out_audit, index=False, encoding="utf-8-sig")

    # Resumen
    print("\n" + "="*60)
    print(f"TOTAL INCLUIDOS (clean): {len(df_clean)} -> {out_clean}")
    print(f"TOTAL EXCLUIDOS (audit): {len(df_audit)} -> {out_audit}")
    print("-"*60)
    print("Conteo mensual (INCLUIDOS):")
    for (yy, mm) in sorted(monthly_counts.keys()):
        print(f"  {yy}-{mm:02d}: {monthly_counts[(yy, mm)]}")
    if not df_audit.empty:
        print("-"*60)
        print("Top motivos de EXCLUSIÓN:")
        print(df_audit["why_excluded"].fillna("").value_counts().head(10).to_string())

if __name__ == "__main__":
    main()


NYT Amazon COMPANY – Rango: 2020-07 .. 2025-07
Procesando 2020-07 …
  -> 2020-07: INCLUIDOS 17
Procesando 2020-08 …
  -> 2020-08: INCLUIDOS 11
Procesando 2020-09 …
  -> 2020-09: INCLUIDOS 11
Procesando 2020-10 …
  -> 2020-10: INCLUIDOS 17
Procesando 2020-11 …
  -> 2020-11: INCLUIDOS 10
Procesando 2020-12 …
  -> 2020-12: INCLUIDOS 7
Procesando 2021-01 …
  -> 2021-01: INCLUIDOS 17
Procesando 2021-02 …
  -> 2021-02: INCLUIDOS 23
Procesando 2021-03 …
  -> 2021-03: INCLUIDOS 32
Procesando 2021-04 …
  -> 2021-04: INCLUIDOS 28
Procesando 2021-05 …
  -> 2021-05: INCLUIDOS 19
Procesando 2021-06 …
  -> 2021-06: INCLUIDOS 20
Procesando 2021-07 …
  -> 2021-07: INCLUIDOS 19
Procesando 2021-08 …
  -> 2021-08: INCLUIDOS 10
Procesando 2021-09 …
  -> 2021-09: INCLUIDOS 14
Procesando 2021-10 …
  -> 2021-10: INCLUIDOS 12
Procesando 2021-11 …
  -> 2021-11: INCLUIDOS 17
Procesando 2021-12 …
  -> 2021-12: INCLUIDOS 11
Procesando 2022-01 …
  -> 2022-01: INCLUIDOS 10
Procesando 2022-02 …
  -> 2022-02: INCLUID

# 2. EXTRACCION DE NOTICIAS EODHD

In [ ]:
# -*- coding: utf-8 -*-
"""
EODHD → AMZN.US  (2020-07-01 .. 2025-07-31)
- Título debe contener 'Amazon' (palabra completa).
- Excluye PR wires, clickbait y entretenimiento/Prime Video.
- Excluye contexto 'Amazon rainforest' salvo señales corporativas.
- Genera resumen extractivo y guarda RAW + CLEAN_SUM.
"""

import json, time, re, requests, pandas as pd
from datetime import datetime, date
from urllib.parse import urlparse

# ------------------ Config ------------------
API_TOKEN = "DEMO"           # DEMO funciona con AMZN.US
TICKER    = "AMZN.US"
PER_PAGE  = 1000
SLEEP_SEC = 0.5
BASE_URL  = "https://eodhd.com/api/news"

DATE_FROM = date(2020, 7, 1)    # rango solicitado
DATE_TO   = date(2025, 7, 31)   # rango solicitado

# ------------------ Descarga ------------------
def fetch_news(ticker: str, dt_from: str, dt_to: str, per_page=PER_PAGE):
    all_rows, offset, calls = [], 0, 0
    while True:
        params = {
            "s": ticker, "from": dt_from, "to": dt_to,
            "limit": per_page, "offset": offset, "fmt": "json",
            "api_token": API_TOKEN
        }
        r = requests.get(BASE_URL, params=params, timeout=30)
        if r.status_code != 200:
            print(f"[ERROR] HTTP {r.status_code} -> {r.text[:300]}")
            r.raise_for_status()
        txt = r.text.strip()
        page = json.loads(txt) if txt else []
        calls += 1
        if not page:
            print(f"[OK] No hay más resultados (offset={offset}).")
            break
        all_rows.extend(page)
        print(f"[OK] offset={offset:>5} | filas={len(page):>4} | llamadas={calls}")
        if len(page) < per_page:
            break
        offset += per_page
        time.sleep(SLEEP_SEC)
    return all_rows, calls

rows, api_calls = fetch_news(TICKER, DATE_FROM.isoformat(), DATE_TO.isoformat())

# ------------------ Flatten ------------------
def flatten(row: dict):
    sent = row.get("sentiment") or {}
    symbols = row.get("symbols") or []
    if isinstance(symbols, str):
        symbols = [x.strip() for x in symbols.split(",") if x.strip()]
    return {
        "date": row.get("date"),
        "title": row.get("title"),
        "content": row.get("content"),
        "link": row.get("link"),
        "source": row.get("source"),
        "lang": row.get("lang"),
        "symbols": ",".join(symbols),
        "tags": ",".join(row.get("tags") or []),
        "polarity": sent.get("polarity"),
        "neg": sent.get("neg"),
        "neu": sent.get("neu"),
        "pos": sent.get("pos"),
    }

df_raw = pd.DataFrame([flatten(x) for x in rows])
safe_from = DATE_FROM.isoformat().replace(":", "-")
safe_to   = DATE_TO.isoformat().replace(":", "-")
raw_path  = f"eodhd_news_{TICKER}_{safe_from}_{safe_to}_RAW.csv"
df_raw.to_csv(raw_path, index=False, encoding="utf-8")
print(f"[RAW] {len(df_raw)} filas -> {raw_path}")

# ------------------ Utilidades de limpieza ------------------
PR_DOMAINS = [
    "globenewswire", "prnewswire", "businesswire", "accesswire",
    "newsfilecorp", "newsdirect", "einnews", "newswire", "prweb",
    "/press-releases", "/pressrelease", "/press-release"
]

CLICKBAIT_PATTERNS = [
    r"\bwhat to know\b", r"\bwhat we know\b", r"\beverything you need to know\b",
    r"\byou won'?t believe\b", r"\bmust[- ]see\b", r"\bbreaking\b", r"\bgoes viral\b",
    r"\btop\s?\d+\b", r"\bbest\b", r"\bhow to watch\b", r"\bwhere to watch\b",
    r"\bstreaming guide\b", r"\brelease date\b", r"\btrailer\b", r"\brecap\b", r"\breview\b",
]
ENTERTAINMENT_PATTERNS = [
    r"\bprime video\b", r"\bseries?\b", r"\bseason\b", r"\bepisode\b",
    r"\bmr\.?\s*&?\s*mrs\.?\s*smith\b", r"\brings?\s*of\s*power\b", r"\breacher\b",
    r"\bouter range\b", r"\bthe boys\b", r"\bexpats\b", r"\bmy lady jane\b", r"\bcast\b"
]
RAINFOREST_BAD = [
    r"\brainforest\b", r"\bdeforestation\b", r"\bdrought\b", r"\bamazon river\b",
    r"\bindigenous\b", r"\btribes?\b"
]
CORP_MARKERS = [r"\bamazon\.com\b", r"\baws\b", r"\bamazon web services\b", r"\bnasdaq:\s*amzn\b"]

def domain_from_link(link: str) -> str:
    try:
        return urlparse(link).netloc.lower()
    except Exception:
        return ""

def has_pr_source(link: str, source: str) -> bool:
    s = f"{link} {source}".lower()
    return any(dom in s for dom in PR_DOMAINS)

def title_has_amazon(title: str) -> bool:
    if not isinstance(title, str): return False
    return bool(re.search(r"\bamazon\b", title, flags=re.I))

def contains_focus_terms(text: str) -> bool:
    if not isinstance(text, str): return False
    text = text.lower()
    return ("amazon" in text) or ("aws" in text)

def norm_title(t: str) -> str:
    if not isinstance(t, str): return ""
    t = t.lower()
    t = re.sub(r"\s+", " ", t)
    t = re.sub(r"[^a-z0-9 ]+", "", t)
    return t.strip()

def is_clickbaity(title: str) -> bool:
    if not isinstance(title, str): return False
    t = title.lower()
    short = len(t) < 28
    bad = any(re.search(p, t) for p in CLICKBAIT_PATTERNS)
    ent = any(re.search(p, t) for p in ENTERTAINMENT_PATTERNS)
    return bad or ent or short

def rainforest_context(text: str) -> bool:
    if not isinstance(text, str): return False
    t = text.lower()
    has_rain = any(re.search(p, t) for p in RAINFOREST_BAD)
    has_corp = any(re.search(p, t) for p in CORP_MARKERS)
    return has_rain and not has_corp

# ------------------ Resumen extractivo (sin APIs) ------------------
_SENT_SPLIT = re.compile(r"(?<!\b[A-Z])(?<=[\.\?\!])\s+(?=[A-Z0-9])")
STOPWORDS = set("""
a about above after again against all am an and any are as at be because been
before being below between both but by can did do does doing down during each few
for from further had has have having he her here hers herself him himself his how
i if in into is it its itself just me more most my myself no nor not of off on
once only or other our ours ourselves out over own same she should so some such
than that the their theirs them themselves then there these they this those through
to too under until up very was we were what when where which while who whom why
with you your yours yourself yourselves
""".split())

def split_sentences(text: str):
    if not isinstance(text, str): return []
    t = re.sub(r"\s+", " ", text).strip().replace("U.S.", "US.")
    return [s.strip() for s in _SENT_SPLIT.split(t) if s.strip()]

def summarize_extract(text: str, max_sents=3, max_chars=420):
    sents = split_sentences(text)
    if not sents: return ""
    words = re.findall(r"[a-zA-Z0-9']+", text.lower())
    freq = {}
    for w in words:
        if w in STOPWORDS or len(w) <= 2: continue
        freq[w] = freq.get(w, 0) + 1
    scored = []
    for i, s in enumerate(sents):
        tokens = re.findall(r"[a-zA-Z0-9']+", s.lower())
        score = sum(freq.get(w, 0) for w in tokens)
        scored.append((i, s, score))
    top = sorted(scored, key=lambda x: x[2], reverse=True)[:max_sents]
    top_sorted = sorted(top, key=lambda x: x[0])
    out, total = [], 0
    for _, s, _ in top_sorted:
        add = ("" if not out else " ") + s
        if total + len(add) > max_chars: break
        out.append(s); total += len(add)
    if not out:
        out = sents[:max_sents]
    return " ".join(out)[:max_chars].rstrip()

# ------------------ Limpieza + filtros ------------------
df = df_raw.copy()
for col in ["title","content","link","source","symbols","tags","lang"]:
    if col not in df.columns: df[col] = ""

df["symbols_list"]  = df["symbols"].apply(lambda s: [x.strip() for x in str(s).split(",") if x.strip()])
df["symbols_count"] = df["symbols_list"].apply(len)
df["has_amzn"]      = df["symbols_list"].apply(lambda xs: "AMZN.US" in xs)
df["title_norm"]    = df["title"].apply(norm_title)
df["content_len"]   = df["content"].map(lambda x: len(str(x)))
df["source_domain"] = df["link"].map(domain_from_link)

mask = True
mask = mask & df["title"].apply(title_has_amazon)                                # título con 'Amazon'
mask = mask & ~df.apply(lambda r: has_pr_source(r["link"], r["source"]), axis=1)  # sin PR wires
mask = mask & df["has_amzn"] & (df["symbols_count"] <= 3)                         # foco en AMZN
mask = mask & (df["content_len"] >= 300)                                          # mínimo texto
mask = mask & (df["title"].apply(lambda t: not is_clickbaity(t)))                 # evita clickbait
mask = mask & (df["content"].apply(lambda t: not rainforest_context(t)))          # fuera selva no corporativa
mask = mask & (df["content"].apply(contains_focus_terms) | (df["content_len"] == 0))

df_f = df[mask].copy()

# Deduplicación
df_f = df_f.sort_values("date").drop_duplicates(subset=["link"], keep="first")
df_f = df_f.sort_values("date").drop_duplicates(subset=["title_norm"], keep="first")

# Resumen
df_f["summary"] = df_f["content"].apply(lambda x: summarize_extract(str(x)))
df_f.loc[df_f["summary"].str.len() == 0, "summary"] = df_f["title"].astype(str).str.slice(0, 240)

# Guardado
keep_cols = [
    "date","title","summary","content","link","source_domain","lang",
    "symbols","tags","polarity","neg","neu","pos","content_len","symbols_count"
]
clean_path = "FINAL_EODHD_NEWS_5_ANIOS.CSV"  
df_f[keep_cols].to_csv(clean_path, index=False, encoding="utf-8")

# ------------------ Recuento por mes ------------------
def to_month(d):
    try:
        return datetime.fromisoformat(str(d).replace("Z","")).strftime("%Y-%m")
    except Exception:
        return ""
df_f["month"] = df_f["date"].apply(to_month)
counts = df_f.groupby("month").size().sort_index()

print("\n=== RECUENTO POR MES (CLEAN) ===")
if len(counts):
    for m, c in counts.items():
        print(f"{m}: {c}")
else:
    print("(vacío)")

print("\n=== RESUMEN ===")
print(f"Descargadas (RAW): {len(df_raw)}")
print(f"Limpias (CLEAN):  {len(df_f)}")
print(f"Llamadas API:     {api_calls}")
print(f"RAW  -> {raw_path}")
print(f"CLEAN-> {clean_path}")


[OK] offset=    0 | filas=1000 | llamadas=1
[OK] offset= 1000 | filas=1000 | llamadas=2
[OK] offset= 2000 | filas=1000 | llamadas=3
[OK] offset= 3000 | filas=1000 | llamadas=4
[OK] offset= 4000 | filas=1000 | llamadas=5
[OK] offset= 5000 | filas=1000 | llamadas=6
[OK] offset= 6000 | filas=1000 | llamadas=7
[OK] offset= 7000 | filas=1000 | llamadas=8
[OK] offset= 8000 | filas=1000 | llamadas=9
[OK] offset= 9000 | filas=1000 | llamadas=10
[OK] offset=10000 | filas=1000 | llamadas=11
[OK] offset=11000 | filas=1000 | llamadas=12
[OK] offset=12000 | filas=1000 | llamadas=13
[OK] offset=13000 | filas=1000 | llamadas=14
[OK] offset=14000 | filas=1000 | llamadas=15
[OK] offset=15000 | filas=1000 | llamadas=16
[OK] offset=16000 | filas=1000 | llamadas=17
[OK] offset=17000 | filas=1000 | llamadas=18
[OK] offset=18000 | filas=1000 | llamadas=19
[OK] offset=19000 | filas=1000 | llamadas=20
[OK] offset=20000 | filas=1000 | llamadas=21
[OK] offset=21000 | filas=1000 | llamadas=22
[OK] offset=22000 |

# 3. EXTRACCION DE NOTICIAS THE GUARDIAN

In [ ]:
# guardian_fetch_amazon.py
import requests, pandas as pd, re, time
from dateutil.relativedelta import relativedelta
from calendar import monthrange
from datetime import date

# Clave eliminada por seguridad para publicación en repositorio
GUARDIAN_API_KEY = "TU_API_KEY_AQUI" 

# Secciones: vacío para máximo recall (filtra luego por contenido)
SECCIONES = []

# Términos buscados solo en titulares
BROAD_TERMS = [
    "amazon","amzn","aws","prime video","alexa","ring","kindle","whole foods",
    "zoox","irobot","mgm","kuiper","project kuiper","prime day","amazon fresh",
    "amazon go","twitch","audible"
]

# Filtros de titular/contenido
RE_STRICT = re.compile(r"\bamazon\b", re.IGNORECASE)
RE_BROAD  = re.compile(
    r"\b(amazon|amzn|aws|prime video|alexa|ring|kindle|whole foods|zoox|irobot|mgm|kuiper|project kuiper|prime day|amazon fresh|amazon go|twitch|audible)\b",
    re.IGNORECASE
)

# Anti-clickbait/entretenimiento
RE_CLICKBAIT = re.compile(
    r"\b(what to know|what we know|everything you need to know|won'?t believe|must[- ]see|breaking|goes viral|"
    r"top\s?\d+|best|how to watch|where to watch|streaming guide|release date|trailer|recap|review|minute[- ]by[- ]minute)\b",
    re.IGNORECASE
)
RE_ENT = re.compile(
    r"\b(prime video|series?|season|episode|cast|the boys|rings? of power|reacher|outer range|expats|mr\.?\s*&?\s*mrs\.?\s*smith)\b",
    re.IGNORECASE
)

# Selva Amazónica (ruido) y marcadores corporativos (permiten)
RE_RAINFOREST = re.compile(r"\b(rainforest|deforestation|drought|amazon river|indigenous|tribes?)\b", re.IGNORECASE)
RE_CORP = re.compile(r"\b(amazon\.com|aws|amazon web services|nasdaq:\s*amzn|earnings|revenue|guidance|nlrb|osha|warehouse|warehouses|fulfillment|prime day|marketplace|advertising|cloud|kuiper|data ?center|ai)\b", re.IGNORECASE)

# Blue Origin solo con Kuiper/satélite Amazon
RE_BLUE_ORIGIN = re.compile(r"\bblue origin\b", re.IGNORECASE)
RE_KUIPER = re.compile(r"\b(kuiper|amazon(?:’|')?s satellite|amazon satellite|project kuiper)\b", re.IGNORECASE)

BASE_URL   = "https://content.guardianapis.com/search"
START_DATE = date(2020, 7, 1)   # 01-07-2020
END_DATE   = date(2025, 7, 31)  # 31-07-2025

def _query_string(terms):
    # Construye query OR con comillas cuando hay espacios
    parts = [f'"{t.strip()}"' if " " in t else t.strip() for t in terms]
    return " OR ".join(parts)

def _bad_entertainment(title, text):
    # Marca entretenimiento/clickbait
    return bool(RE_CLICKBAIT.search(title) or RE_ENT.search(title) or RE_ENT.search(text))

def _rainforest_without_corp(title, text):
    # Selva sin marcadores corporativos
    t = f"{title}\n{text}"
    return bool(RE_RAINFOREST.search(t) and not RE_CORP.search(t))

def _blue_origin_without_kuiper(title, text):
    # Blue Origin sin Kuiper/satélite Amazon
    t = f"{title}\n{text}"
    return bool(RE_BLUE_ORIGIN.search(t) and not RE_KUIPER.search(t))

def fetch_guardian():
    if not GUARDIAN_API_KEY or GUARDIAN_API_KEY == "TU_API_KEY_AQUI":
        raise SystemExit("❌ Añade tu API key en GUARDIAN_API_KEY.")

    q = _query_string(BROAD_TERMS)

    rows = []
    cursor = pd.Timestamp(START_DATE.year, START_DATE.month, 1)
    endm   = pd.Timestamp(END_DATE.year, END_DATE.month, 1)

    # Recorre meses del rango
    while cursor <= endm:
        y, m = cursor.year, cursor.month
        from_date = f"{y}-{m:02d}-01"
        to_date   = f"{y}-{m:02d}-{monthrange(y, m)[1]:02d}"

        secciones = SECCIONES if SECCIONES else [None]
        total_mes = 0

        for sec in secciones:
            page, pages = 1, 1
            while page <= pages:
                params = {
                    "api-key": GUARDIAN_API_KEY,
                    "q": q,                              # términos
                    "query-fields": "headline",          # solo titulares
                    "type": "article",                   # fuera liveblogs
                    "from-date": from_date,
                    "to-date": to_date,
                    "page-size": 200,
                    "page": page,
                    "order-by": "newest",
                    "use-date": "published",
                    "show-fields": "headline,trailText,standfirst,bodyText"
                }
                if sec:
                    params["section"] = sec

                try:
                    r = requests.get(BASE_URL, params=params, timeout=30)
                    r.raise_for_status()
                except requests.RequestException as e:
                    print(f"  ! Error {cursor.strftime('%Y-%m')} pág {page}: {e}")
                    break

                resp  = r.json().get("response", {})
                pages = resp.get("pages", 1)
                res   = resp.get("results", [])

                if page == 1:
                    sec_txt = sec if sec else "ALL"
                    print(f"{cursor.strftime('%Y-%m')} [{sec_txt}] → total={resp.get('total',0)} pág(s)={pages}")

                for it in res:
                    title = it.get("webTitle","") or ""
                    text  = it.get("fields",{}).get("bodyText","") or ""
                    rows.append({
                        "Fecha":   it.get("webPublicationDate"),
                        "Seccion": it.get("sectionName"),
                        "Titular": title,
                        "URL":     it.get("webUrl","") or "",
                        "Texto":   text
                    })
                    total_mes += 1

                page += 1
                time.sleep(0.12)  # cortesía

        print(f"  ↳ acumulados {cursor.strftime('%Y-%m')}: {total_mes}")
        cursor += relativedelta(months=1)

    if not rows:
        raise SystemExit("Sin resultados.")

    # Limpieza y rango fijo
    df = pd.DataFrame(rows).drop_duplicates(subset=["URL"])
    df["Fecha"] = pd.to_datetime(df["Fecha"], utc=True).dt.tz_convert(None)
    df = df[(df["Fecha"] >= pd.Timestamp(START_DATE)) & (df["Fecha"] <= pd.Timestamp(END_DATE))]

    # Filtro principal: titular debe contener 'amazon' exacto
    df = df[df["Titular"].str.contains(RE_STRICT, na=False)]
    # También exige presencia de términos ampliados en titular
    df = df[df["Titular"].str.contains(RE_BROAD, na=False)]

    # Filtros negativos (menos ruido)
    df = df[~df.apply(lambda r: _bad_entertainment(r["Titular"], r["Texto"]), axis=1)]
    df = df[~df.apply(lambda r: _rainforest_without_corp(r["Titular"], r["Texto"]), axis=1)]
    df = df[~df.apply(lambda r: _blue_origin_without_kuiper(r["Titular"], r["Texto"]), axis=1)]

    # Orden final
    df = df.sort_values("Fecha", ascending=False)

    # Guardado único con el nombre solicitado
    out_path = "FINAL_THE_GUARDIAN_NEWS_5_ANIOS.CSV"
    df.to_csv(out_path, index=False, encoding="utf-8-sig")

    # Resumen
    print("\n=== RESUMEN ===")
    print(f"Total únicos (filtrados): {len(df):,}")
    print(f"Guardado: {out_path}")

if __name__ == "__main__":
    fetch_guardian()


2020-07 [ALL] → total=13 pág(s)=1
  ↳ acumulados 2020-07: 13
2020-08 [ALL] → total=17 pág(s)=1
  ↳ acumulados 2020-08: 17
2020-09 [ALL] → total=12 pág(s)=1
  ↳ acumulados 2020-09: 12
2020-10 [ALL] → total=21 pág(s)=1
  ↳ acumulados 2020-10: 21
2020-11 [ALL] → total=19 pág(s)=1
  ↳ acumulados 2020-11: 19
2020-12 [ALL] → total=14 pág(s)=1
  ↳ acumulados 2020-12: 14
2021-01 [ALL] → total=13 pág(s)=1
  ↳ acumulados 2021-01: 13
2021-02 [ALL] → total=18 pág(s)=1
  ↳ acumulados 2021-02: 18
2021-03 [ALL] → total=17 pág(s)=1
  ↳ acumulados 2021-03: 17
2021-04 [ALL] → total=19 pág(s)=1
  ↳ acumulados 2021-04: 19
2021-05 [ALL] → total=28 pág(s)=1
  ↳ acumulados 2021-05: 28
2021-06 [ALL] → total=24 pág(s)=1
  ↳ acumulados 2021-06: 24
2021-07 [ALL] → total=14 pág(s)=1
  ↳ acumulados 2021-07: 14
2021-08 [ALL] → total=9 pág(s)=1
  ↳ acumulados 2021-08: 9
2021-09 [ALL] → total=17 pág(s)=1
  ↳ acumulados 2021-09: 17
2021-10 [ALL] → total=20 pág(s)=1
  ↳ acumulados 2021-10: 20
2021-11 [ALL] → total=21 p

C:\Users\gabri\AppData\Local\Temp\ipykernel_40324\1327233339.py:150: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df["Titular"].str.contains(RE_BROAD, na=False)]



=== RESUMEN ===
Total únicos (filtrados): 349
Guardado: FINAL_THE_GUARDIAN_NEWS_5_ANIOS.CSV


# 4. Unificar dataset

In [ ]:
# -*- coding: utf-8 -*-
# Une los tres CSV de noticias (EODHD, NYT, The Guardian) en un único fichero normalizado.
# Requisitos: pandas (pip install pandas)

from pathlib import Path
import re
import pandas as pd

# --- Configuración mínima ---
RUTA_BASE = Path(".")  
ARCHIVOS = [
    RUTA_BASE / "FINAL_EODHD_NEWS_5_ANIOS.CSV",
    RUTA_BASE / "FINAL_NYT_NEWS_5_ANIOS.CSV",
    RUTA_BASE / "FINAL_THE_GUARDIAN_NEWS_5_ANIOS.CSV",
]
SALIDA = RUTA_BASE / "NOTICIAS_UNIFICADAS_5_ANIOS.csv"

CANDIDATOS_FECHA = [
    "Fecha","fecha","date","published","pub_date","publication_date",
    "webPublicationDate","time","created_at","created_utc","datetime"
]
CANDIDATOS_TITULO = [
    "Titular","titular","title","headline.main","headline_main","headline",
    "webTitle","heading","name"
]
CANDIDATOS_RESUMEN = [
    "resumen","summary","abstract","snippet","lead_paragraph","trailText",
    "standfirst","description","dek"
]

def _leer_csv(carpeta_o_fichero: Path) -> pd.DataFrame:
    """Se intenta con varias codificaciones; se delega el resto a pandas."""
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1"):
        try:
            return pd.read_csv(carpeta_o_fichero, encoding=enc)
        except UnicodeDecodeError:
            continue
  
    return pd.read_csv(carpeta_o_fichero, engine="python")

def _col(df: pd.DataFrame, candidatos: list[str]) -> str | None:
    """Se localiza la primera columna existente según una lista de candidatos (case-insensitive)."""
    mapa = {c.lower(): c for c in df.columns}
    # Búsqueda exacta (insensible a mayúsculas)
    for c in candidatos:
        if c.lower() in mapa:
            return mapa[c.lower()]
    # Búsqueda por inclusión (por si viene con prefijos/sufijos inesperados)
    for c in candidatos:
        for k in mapa:
            if c.lower() in k:
                return mapa[k]
    return None

_TAGS = re.compile(r"<[^>]+>")
def _limpiar_html(x: str) -> str:
    """Se eliminan etiquetas HTML básicas en resúmenes/entradillas."""
    if not isinstance(x, str):
        return ""
    return _TAGS.sub("", x).strip()

def _normalizar(df: pd.DataFrame) -> pd.DataFrame:
    """Se normaliza a columnas: Fecha (ISO), Titular, resumen."""
    c_fecha = _col(df, CANDIDATOS_FECHA)
    c_tit   = _col(df, CANDIDATOS_TITULO)
    c_res   = _col(df, CANDIDATOS_RESUMEN)

    # Se crean columnas seguras aunque falten en origen
    tmp = pd.DataFrame()
    if c_fecha is None:
        # Sin fecha no se puede integrar: se descarta en bloque
        return tmp

    tmp["Fecha"] = pd.to_datetime(df[c_fecha], errors="coerce", utc=True)

    if c_tit is not None:
        tmp["Titular"] = df[c_tit].astype(str).str.strip()
    else:
        tmp["Titular"] = ""

    if c_res is not None:
        tmp["resumen"] = df[c_res].astype(str).map(_limpiar_html)
    else:
        tmp["resumen"] = ""

    # Se filtra lo no convertible a fecha y filas totalmente vacías de texto
    tmp = tmp.dropna(subset=["Fecha"])
    vacias = (tmp["Titular"].str.len() == 0) & (tmp["resumen"].str.len() == 0)
    tmp = tmp[~vacias]

    return tmp

def main():
    # Se cargan, normalizan y apilan las tres fuentes heterogéneas
    marcos = []
    for f in ARCHIVOS:
        df_raw = _leer_csv(f)
        marcos.append(_normalizar(df_raw))

    df = pd.concat(marcos, ignore_index=True)

    # Se ordena por fecha y se eliminan duplicados evidentes
    df = df.sort_values("Fecha").drop_duplicates(subset=["Fecha", "Titular", "resumen"])

    # Se exporta con fecha en ISO 8601 (manteniendo hora si existe)
    df["Fecha"] = df["Fecha"].dt.strftime("%Y-%m-%d %H:%M:%S")
    df = df[["Fecha", "Titular", "resumen"]]

    df.to_csv(SALIDA, index=False, encoding="utf-8-sig")
    print(f"Listo: {SALIDA.resolve()}  ({len(df):,} filas)")

if __name__ == "__main__":
    main()


Listo: C:\Users\gabri\OneDrive\Desktop\Desktop\UNIR\TFM\Codigo\NOTICIAS_UNIFICADAS_5_ANIOS.csv  (2,469 filas)


LIMPIEZA MANUAL: Amazonas, clickbait, eliminadas las que no tienen resumen

Van 2051

NOTICIAS_UNIFICADAS_5_ANIOS - V2

Codigo siguiente marca relevantes: 

“Si” (relevante para AMZN): resultados trimestrales, guidance, márgenes/KPIs (p. ej., AWS), M&A/ventas de activos, cambios de precios (Prime/AWS), grandes contratos/alianzas, outages o incidentes de seguridad en AWS, decisiones regulatorias (FTC/UE), cambios directivos o despidos masivos, huelgas/logística con impacto amplio.

“No” (no mueve la cotización): noticias del Amazonas/Amazonía (selva/región), eventos locales/menores (una nave, una ciudad, una acción benéfica), clickbait/tutoriales/cupones/reseñas, notas genéricas del sector sin vínculo material con Amazon, resúmenes duplicados o sin novedad, títulos vagos o sin resumen.

Ambigüas → criterio conservador: si no queda claro el impacto financiero o no se menciona Amazon/AWS/Prime/negocios asociados de forma explícita, se marca “No”.

Idea-fuerza: solo se etiqueta “Si” cuando el contenido podría mover el precio o refleja cambios materiales en el negocio de Amazon.

In [ ]:
# -*- coding: utf-8 -*-
# Clasifica relevancia para AMZN en TODO el dataset y añade 'Relevante' (Si/No) a cada fila.
# Requisitos: pip install openai pandas

import os, re, json, time, hashlib
from pathlib import Path
import pandas as pd
from openai import OpenAI

# --- Configuración ---
INPUT_FILE  = Path("NOTICIAS_UNIFICADAS_5_ANIOS - V2.csv")
OUTPUT_FILE = INPUT_FILE.with_name(f"{INPUT_FILE.stem} - con Relevante.csv")
MODEL = "gpt-4o-mini"
MAX_OUTPUT_TOKENS = 8
SAVE_EVERY = 500  # guarda progreso cada X filas
# Clave eliminada por seguridad para publicación en repositorio
API_KEY = "TU_API_KEY_AQUI"
client = OpenAI(api_key=API_KEY)

# --- Reglas fijas (prefijo estable para aprovechar cache del servidor) ---
SYSTEM_PROMPT = """
You are a finance/news relevance classifier. Decide if a news item is relevant to Amazon.com, Inc.’s stock (AMZN).
Output EXACTLY one JSON: {"Relevante":"Si"} or {"Relevante":"No"} — in Spanish, initial capital S/N, no extra fields.

Label “Si” if it plausibly moves AMZN or reflects material fundamentals/risks:
- Earnings, revenue, guidance, KPIs (AWS growth/margins), buybacks/dividends.
- Major AWS events (outages, security, pricing, large client wins/losses, AI/infra with clear scale).
- M&A/divestitures/investments (e.g., Project Kuiper milestones with commercial impact).
- Government/regulatory/legal actions materially involving Amazon (FTC/DoJ/EU, big fines, union rulings, targeted taxation).
- Leadership/board changes, large layoffs/hiring waves, strategy shifts.
- Prime/retail logistics with clear financial impact (Prime price change, nationwide strikes, broad delivery disruptions).
- Large partnerships/contracts with quantified or clearly material scope.

Label “No” if:
- It is about the Amazon rainforest/region (“Amazonas”, “Amazonía”, “Manaus”, deforestation, indigenous communities…).
- Local/minor items (single warehouse accidents, local charity/store opening) without broad financial impact.
- Clickbait, listicles, coupons, how-to guides, product reviews, marketing fluff, celebrity gossip.
- Generic industry news without specific material link to Amazon.
- Duplicate/trivial updates with no new material info.
- Missing summary and vague title with no material signal.

Be conservative: default to “No” unless materiality is clear.

INPUT:
Title: <string>
Summary: <string or empty>

OUTPUT (strict):
{"Relevante":"Si"}  OR  {"Relevante":"No"}
""".strip()

# --- Prefiltro local (reduce llamadas obvias) ---
RE_RAINFOREST = re.compile(r"\b(amazonas|amazonía|amazônia|manaus|rainforest|amazon basin|amaz[oó]n.+forest)\b", re.I)
RE_CLICKBAIT  = re.compile(r"\b(how to|guide|coupon|deal|promo|trick|tips|hacks|ranking|top \d+|you won.?t believe)\b", re.I)
RE_LOCAL      = re.compile(r"\b(local|community|neighborhood|parish|county fair|charity 5k|school fundraiser)\b", re.I)
RE_COMPANY    = re.compile(
    r"\b(amazon\.com|amzn|amazon|aws|prime( video)?|whole foods|ring|twitch|kindle|zoox|irobot|mgm|audible|kuiper|project kuiper|amazon go|amazon fresh)\b",
    re.I,
)

def cheap_prefilter(title: str, summary: str) -> str | None:
    """Marca 'No' si es irrelevante evidente; None si es dudoso."""
    t = (title or "").strip()
    s = (summary or "").strip()
    txt = f"{t} {s}"
    if RE_RAINFOREST.search(txt): return "No"
    if RE_CLICKBAIT.search(txt):  return "No"
    if RE_LOCAL.search(txt):      return "No"
    if not RE_COMPANY.search(txt):# si no menciona la compañía/entorno directo
        return "No"
    if not t and not s:
        return "No"
    return None

# --- Cache local por duplicados (evita pagar dos veces por el mismo texto) ---
def key_for(title: str, summary: str) -> str:
    h = hashlib.sha256(f"{title}||{summary}".encode("utf-8")).hexdigest()
    return h

def classify_with_openai(title: str, summary: str, retry=3) -> str:
    """Llama a OpenAI con salida JSON estricta y devuelve 'Si'/'No'."""
    for attempt in range(retry):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                temperature=0,
                max_tokens=MAX_OUTPUT_TOKENS,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Title: {title}\nSummary: {summary or ''}"}
                ],
            )
            content = (resp.choices[0].message.content or "").strip()
            data = json.loads(content) if content.startswith("{") else {}
            val = data.get("Relevante", "No")
            return "Si" if val == "Si" else "No"
        except Exception:
            time.sleep(1.5 * (attempt + 1))
    return "No"

def read_csv_any(path: Path) -> pd.DataFrame:
    """Lee el CSV completo preservando columnas."""
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, engine="python")

def main():
    df = read_csv_any(INPUT_FILE).copy()
    col_fecha   = "Fecha"   if "Fecha"   in df.columns else df.columns[0]
    col_titular = "Titular" if "Titular" in df.columns else df.columns[1]
    col_resumen = "resumen" if "resumen" in df.columns else df.columns[2]

    if "Relevante" not in df.columns:
        df["Relevante"] = ""

    cache: dict[str, str] = {}
    api_calls = 0

    for i, row in df.iterrows():
        if df.at[i, "Relevante"]:
            continue  # respeta valores ya existentes si relanzas

        title = str(row.get(col_titular, "") or "")
        summ  = str(row.get(col_resumen, "") or "")

        # Prefiltro
        pre = cheap_prefilter(title, summ)
        if pre is not None:
            df.at[i, "Relevante"] = pre
        else:
            # Cache por contenido
            k = key_for(title, summ)
            if k in cache:
                df.at[i, "Relevante"] = cache[k]
            else:
                api_calls += 1
                label = classify_with_openai(title, summ)
                cache[k] = label
                df.at[i, "Relevante"] = label

        # Guardado incremental
        if (i + 1) % SAVE_EVERY == 0:
            df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
            print(f"[{i+1}/{len(df)}] guardado parcial… (llamadas API: {api_calls})")

    # Guardado final
    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    si = int((df["Relevante"] == "Si").sum())
    no = int((df["Relevante"] == "No").sum())
    print(f"Guardado: {OUTPUT_FILE.resolve()}")
    print(f"Filas: {len(df)} | Si={si} No={no} | Llamadas a API: {api_calls}")

if __name__ == "__main__":
    main()


[500/2048] guardado parcial… (llamadas API: 462)
[1000/2048] guardado parcial… (llamadas API: 902)
[1500/2048] guardado parcial… (llamadas API: 1337)
[2000/2048] guardado parcial… (llamadas API: 1809)
Guardado: C:\Users\gabri\OneDrive\Desktop\Desktop\UNIR\TFM\Codigo\NOTICIAS_UNIFICADAS_5_ANIOS - V2 - con Relevante.csv
Filas: 2048 | Si=1201 No=847 | Llamadas a API: 1854


Introduzco sentimiento

In [ ]:
# -*- coding: utf-8 -*-
# Puntúa –10..+10 el impacto para AMZN en las primeras N_ROWS noticias.
# Requisitos: pip install openai pandas

import re, json, time, sys, traceback
from pathlib import Path
import pandas as pd
from openai import OpenAI

# =============== CONFIG =================
INPUT_FILE  = Path("NOTICIAS_UNIFICADAS_5_ANIOS - V3.csv")
OUTPUT_FILE = INPUT_FILE.with_name(f"{INPUT_FILE.stem} - puntuado_200.csv")
N_ROWS = None           # pon None para TODO el dataset
MODEL = "gpt-4o-mini"
MAX_TOKENS = 8         # solo necesitamos un entero corto

# Clave eliminada por seguridad para publicación en repositorio
API_KEY = "TU_API_KEY_AQUI"
client = OpenAI(api_key=API_KEY)

# ===== Prompt (sin JSON): devuelve SOLO un entero -10..10 =====
SYSTEM_PROMPT = """
You are a financial impact rater for Amazon.com, Inc. (AMZN).
Return ONLY one integer from -10 to +10 (no words). Positive = good for AMZN; negative = bad.
Bigger |score| = more material (magnitude, breadth/duration, directness to AMZN/AWS/Prime/Whole Foods/Kuiper, novelty, credibility).

Scale (guidance):
+9..+10 blockbuster beat/guidance raise; multi-billion contract; antitrust case dismissed; big buyback/dividend.
+6..+8  clear beat; AWS acceleration; major partnership/pricing with numbers; accretive M&A.
+3..+5  moderate positive; some numbers/scope.
+1..+2  mild/uncertain positive.
 0       neutral/ambiguous; rainforest/region “Amazonas/Amazonía/Manaus”.
-1..-2  mild/uncertain negative.
-3..-5  miss/guidance trim; sizable fine; regional strike; slowdown datapoint.
-6..-8  major negative: AWS outage/security; antitrust/regulatory action; broad labor disruption; big breach; guidance cut.
-9..-10 severe negative: multi-region/multi-day outage; lost landmark case with heavy remedies; breakup-like remedy; huge recurring costs/taxation.
""".strip()
# =======================================

def read_csv_any(path: Path, nrows=None) -> pd.DataFrame:
    """Lee con fallback de codificación."""
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc, nrows=nrows)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, engine="python", nrows=nrows)

def usage_to_dict(usage) -> dict:
    if not usage:
        return {}
    d = {}
    for k in ("prompt_tokens", "completion_tokens", "total_tokens"):
        v = getattr(usage, k, None)
        if v is not None:
            d[k] = v
    return d

def score_with_openai(title: str, summary: str, retry: int = 3):
    """
    Devuelve (puntuacion:int, usage:dict). En fallo, (0, {}).
    Sin JSON; se extrae el primer entero de la respuesta.
    """
    last_err = None
    for attempt in range(retry):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                temperature=0,
                max_tokens=MAX_TOKENS,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Title: {title}\nSummary: {summary or ''}\nAnswer with a single integer between -10 and 10. Return only the number."}
                ],
            )
            content = (resp.choices[0].message.content or "").strip()
            usage = usage_to_dict(getattr(resp, "usage", None))

            m = re.search(r"-?\d+", content)  # primer entero
            val = int(m.group()) if m else 0
            val = max(-10, min(10, val))      # saturación a rango
            return val, usage
        except Exception as e:
            last_err = e
            time.sleep(1.2 * (attempt + 1))
    print("[WARN] 0 por fallo tras reintentos. Último error:")
    traceback.print_exception(type(last_err), last_err, None)
    return 0, {}

def main():
    # Carga parcial o total
    df = read_csv_any(INPUT_FILE, nrows=N_ROWS).copy() if N_ROWS else read_csv_any(INPUT_FILE).copy()

    # Columnas esperadas
    col_titular = "Titular" if "Titular" in df.columns else df.columns[1]
    col_resumen = "resumen" if "resumen" in df.columns else df.columns[2]

    if "Puntuacion" not in df.columns:
        df["Puntuacion"] = ""

    total_prompt = total_completion = 0

    for i, row in df.iterrows():
        title = str(row.get(col_titular, "") or "")
        summ  = str(row.get(col_resumen, "") or "")
        val, usage = score_with_openai(title, summ)
        df.at[i, "Puntuacion"] = val
        total_prompt     += usage.get("prompt_tokens", 0)
        total_completion += usage.get("completion_tokens", 0)

        if (i + 1) % 25 == 0:
            print(f"Progreso real: {i+1}/{len(df)} | tokens prompt={total_prompt} completion={total_completion}")

    # Guarda (igual al original + columna nueva)
    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f"Guardado: {OUTPUT_FILE.resolve()}")
    print(f"Tokens usados: prompt={total_prompt}, completion={total_completion}, total={total_prompt+total_completion}")

if __name__ == "__main__":
    main()


Progreso real: 25/1201 | tokens prompt=8953 completion=46
Progreso real: 50/1201 | tokens prompt=17814 completion=91
Progreso real: 75/1201 | tokens prompt=27345 completion=128
Progreso real: 100/1201 | tokens prompt=37102 completion=172
Progreso real: 125/1201 | tokens prompt=46588 completion=218
Progreso real: 150/1201 | tokens prompt=56376 completion=263
Progreso real: 175/1201 | tokens prompt=65889 completion=303
Progreso real: 200/1201 | tokens prompt=75642 completion=340
Progreso real: 225/1201 | tokens prompt=85446 completion=376
Progreso real: 250/1201 | tokens prompt=95122 completion=414
Progreso real: 275/1201 | tokens prompt=104887 completion=440
Progreso real: 300/1201 | tokens prompt=114694 completion=481
Progreso real: 325/1201 | tokens prompt=124528 completion=516
Progreso real: 350/1201 | tokens prompt=134059 completion=555
Progreso real: 375/1201 | tokens prompt=143915 completion=588
Progreso real: 400/1201 | tokens prompt=153533 completion=629
Progreso real: 425/1201 

Ahora hago la media de cada dia

In [ ]:
# -*- coding: utf-8 -*-
# Media diaria de 'Puntuacion' alineada a días de mercado (ET, cierre→cierre 16:00).
# Entrada: NOTICIAS_UNIFICADAS_5_ANIOS - V3 - puntuado.csv
# Salida:  NOTICIAS_UNIFICADAS_5_ANIOS - V3 - puntuado - diario_media.csv

import pandas as pd
from pathlib import Path
from datetime import timedelta, date

INPUT_FILE  = Path("NOTICIAS_UNIFICADAS_5_ANIOS - V3 - puntuado.csv")
OUTPUT_FILE = INPUT_FILE.with_name(f"{INPUT_FILE.stem} - diario_media.csv")

ET_TZ = "America/New_York"
CLOSE_H, CLOSE_M = 16, 0  

def read_csv_any(p: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(p, engine="python")

def push_to_next_weekday(d: date) -> date:
    while d.weekday() >= 5:  # 5=sábado, 6=domingo
        d = d + timedelta(days=1)
    return d

def main():
    df = read_csv_any(INPUT_FILE).copy()

    # 1) Parseo de fecha-hora. El fichero viene en ET ya (naive), formato dd-mm-yy HH:MM.
    dt1 = pd.to_datetime(df["Fecha"], format="%d-%m-%y %H:%M", errors="coerce")
    dt2 = pd.to_datetime(df.loc[dt1.isna(), "Fecha"], errors="coerce", dayfirst=True)  # fallback
    dt  = dt1.fillna(dt2)
    # Localizar a ET (no convertir: las horas ya son ET)
    dt  = dt.dt.tz_localize(ET_TZ, nonexistent="shift_forward", ambiguous="infer")

    # 2) Puntuación válida
    df["Puntuacion"] = pd.to_numeric(df["Puntuacion"], errors="coerce")
    m = dt.notna() & df["Puntuacion"].notna()
    df, dt = df.loc[m].copy(), dt.loc[m]

    # 3) Día de mercado (cierre→cierre)
    day   = dt.dt.floor("D")
    close = day + pd.to_timedelta(f"{CLOSE_H:02d}:{CLOSE_M:02d}:00")
    after = dt >= close

    market_day = day.dt.date
    market_day = market_day.where(~after, market_day + pd.to_timedelta(1, unit="D"))
    market_day = market_day.apply(lambda d: push_to_next_weekday(pd.to_datetime(d).date()))
    df["Market_Day"] = pd.to_datetime(market_day)

    # 4) Agregación
    out = (
        df.groupby("Market_Day", as_index=False)["Puntuacion"]
          .mean()
          .rename(columns={"Market_Day":"Fecha", "Puntuacion":"Media_Puntuacion"})
          .sort_values("Fecha")
          .reset_index(drop=True)
    )
    out["Fecha"] = pd.to_datetime(out["Fecha"]).dt.strftime("%Y-%m-%d")
    out["Media_Puntuacion"] = out["Media_Puntuacion"].round(3)
    out[["Fecha","Media_Puntuacion"]].to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f"Guardado: {OUTPUT_FILE.resolve()}  |  Filas: {len(out)}")

    # 5) Chequeo rápido
    chk = out.set_index("Fecha").loc[["2025-07-28","2025-07-29","2025-07-30","2025-07-31","2025-08-01"], "Media_Puntuacion"]
    print("\nComprobación (esperado 3, 7, 4, 2.4, 3.286):")
    print(chk.to_string())

if __name__ == "__main__":
    main()


Guardado: C:\Users\gabri\OneDrive\Desktop\Desktop\UNIR\TFM\Codigo\NOTICIAS_UNIFICADAS_5_ANIOS - V3 - puntuado - diario_media.csv  |  Filas: 436

Comprobación (esperado 3, 7, 4, 2.4, 3.286):
Fecha
2025-07-28    1.667
2025-07-29    7.000
2025-07-30    3.000
2025-07-31    5.000
2025-08-01    2.688


Añado la columna de sentimiento a mi dataset grande rellenando con 0 las q no tienen

In [27]:
# Añadir columna de sentimiento diario SIN CAMBIAR NADA del CSV original
# - Lee features como TEXTO para preservar formato (fechas, decimales, etc.)
# - Une por fecha usando una clave interna (sin modificar el CSV)
# - Agrega 'news_sent_mean' al final; días sin noticia -> 0
# Requisitos: pandas

import pandas as pd
from pathlib import Path

FEAT_CSV = Path("AMZN_features_10y_basicos_plus_options.csv")
NEWS_CSV = Path("NOTICIAS_UNIFICADAS_5_ANIOS - V3 - puntuado - diario_media.csv")
OUT_CSV  = Path("AMZN_features_10y_basicos_plus_options_with_news.csv")

# --- Helpers ---
def pick_date_col(df):
    # Busca una columna de fecha por nombre típico; si no, prueba por parseo
    cand_names = {"date","fecha","datetime","day","dt","time","timestamp"}
    for c in df.columns:
        if c.strip().lower() in cand_names:
            return c
    # si no, la que mejor se convierta a fecha
    best, best_ok = None, 0
    for c in df.columns:
        ok = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True).notna().mean()
        if ok > best_ok:
            best_ok, best = ok, c
    if best is None or best_ok < 0.5:
        raise ValueError("No pude identificar la columna de fecha.")
    return best

def parse_dates_best(s):
    # Intenta dos interpretaciones y elige la que tenga más válidos
    d1 = pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    d2 = pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True)
    d = d1 if d1.notna().sum() >= d2.notna().sum() else d2
    return d.dt.normalize()

# --- 1) Leer features COMO TEXTO (preserva formato exacto) ---
feat_txt = pd.read_csv(FEAT_CSV, dtype=str, keep_default_na=False)  # no convierte NaN a float
date_col_feat = pick_date_col(feat_txt)

# Clave interna de fechas para alinear (no se guarda en el CSV)
feat_key = parse_dates_best(feat_txt[date_col_feat])

# --- 2) Leer noticias (numérico normal) y preparar media diaria ---
news = pd.read_csv(NEWS_CSV)
date_col_news = pick_date_col(news)
news[date_col_news] = parse_dates_best(news[date_col_news])

SENT_COL = "Media_Puntuacion"
if SENT_COL not in news.columns:
    raise ValueError(f"No se encontró la columna {SENT_COL} en el CSV de noticias.")

news_daily = (
    news.dropna(subset=[date_col_news])[ [date_col_news, SENT_COL] ]
        .groupby(date_col_news, as_index=True)[SENT_COL]
        .mean()
)

# --- 3) Generar la nueva columna alineada a las fechas del CSV de features ---
new_col = feat_key.map(news_daily).fillna(0.0)  # 0 si no hay noticia ese día

# Mantener el CSV idéntico y añadir columna al final
feat_out = feat_txt.copy()
feat_out["news_sent_mean"] = new_col  # solo se añade esta columna

# --- 4) Guardar (sin índice, mismo delimitador) ---
feat_out.to_csv(OUT_CSV, index=False)
print(f"OK -> {OUT_CSV} | filas = {len(feat_out)} | nueva columna: 'news_sent_mean'")


OK -> AMZN_features_10y_basicos_plus_options_with_news.csv | filas = 2315 | nueva columna: 'news_sent_mean'


C:\Users\gabri\AppData\Local\Temp\ipykernel_40324\343269838.py:33: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  d1 = pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_40324\343269838.py:34: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  d2 = pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_40324\343269838.py:33: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict vers

Añado features derivadas de las noticias

In [2]:
import pandas as pd

# === Config ===
FILE_IN  = "AMZN_features_10y_basicos_plus_options_with_news.csv"
FILE_OUT = "AMZN_features_10y_basicos_plus_options_with_news_feats.csv"
NEWS_COL = "news_sent_mean"

# Ventanas por modelo
WINDOWS = {
    "mc_1d": 3,    # muy corto
    "c_21d": 21,   # corto
    "m_252d": 63,  # medio
    "l_504d": 126  # largo
}

# === Carga (NO ordenar, NO tocar nada) ===
df = pd.read_csv(FILE_IN)

# Serie de trabajo (NO modifica la columna original en df)
s = pd.to_numeric(df[NEWS_COL], errors="coerce").fillna(0.0)

# === Añadir SOLO las nuevas columnas al final ===
for tag, w in WINDOWS.items():
    df[f"news_mean_w{w}"] = s.rolling(window=w, min_periods=w).mean()
    df[f"news_std_w{w}"]  = s.rolling(window=w, min_periods=w).std(ddof=0)

# === Guardar (mismo orden de filas y columnas previas intactas) ===
df.to_csv(FILE_OUT, index=False)
